In [1]:
import cv2
from skimage import io, img_as_float
from skimage.filters import rank
from skimage.morphology import disk
import numpy as np

face_results = []
keyp=[]

sift = cv2.xfeatures2d.SIFT_create()
def detect_faces(r):
    image = cv2.imread(r)
    keypoints, descriptors = sift.detectAndCompute(image, None)
    keyp=len(keypoints)
    return keyp

def evaluate_resolucion(a):
    img = cv2.imread(a)
    height, width, channels = img.shape
    if height < 64 or width < 64:
        return True
    return False

def evaluate_brillo(a):
    img = cv2.imread(a)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    mean_brightness = v.mean()
    return mean_brightness

def evaluate_contraste(a):
    img = cv2.imread(a)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mean_contrast = cv2.Laplacian(gray, cv2.CV_64F).var()
    return mean_contrast

def evaluate_entropia(a):
    im1 = img_as_float(io.imread(a, as_gray=True))
    radius = 15
    selem = disk(radius)
    entropy = rank.entropy(im1, selem=selem)
    mean_entropy = np.mean(entropy)
    return mean_entropy

def difuminacion(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
    return laplacian_var

In [2]:
import cv2
from skimage import io, img_as_float
from skimage.filters import rank
from skimage.morphology import disk
import numpy as np
lista_fallas=[]
imagenes=[]
carpeta_base1=[]
n1=[]
puntosdet=[]
brillo=[]
contr=[]
reso=[]
entro=[]
difu=[]
####
def evaluate_image_qualitys(a):
    if evaluate_resolucion(a):
        return False
    if evaluate_brillo(a) < 14:
        return False
    if evaluate_contraste(a) < 3.5:
        return False
    if evaluate_entropia(a) < 3:
        return False
    if detect_faces(a)<=3:
        return False
    if  difuminacion(a) < 5.7:
        return False
    return True

In [3]:
import cv2
import skimage
from skimage.feature import hog
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def sim_coseno(ruta_imagen1, ruta_imagen2):
    image1 = skimage.io.imread(ruta_imagen1)
    image2 = skimage.io.imread(ruta_imagen2)

    # Obtener los descriptores de características HOG
    fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
                          cells_per_block=(1, 1), visualize=True, multichannel=True, channel_axis=2)
    fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
                          cells_per_block=(1, 1), visualize=True, multichannel=True, channel_axis=2)
    max_dim = max(fd1.shape[0], fd2.shape[0])
    fd1 = np.resize(fd1, (max_dim,))
    fd2 = np.resize(fd2, (max_dim,))
    similitud = cosine_similarity(fd1.reshape(1, -1), fd2.reshape(1, -1))
    
    return similitud[0][0]

In [4]:
# PARA LA DETECCION DE FALLAS
import os
import pandas as pd
import numpy as np
#MODIFICAR TAMBIEN EL TO_CSV O SINO SE REESCRIBIRÁ!!!!!!!!!!!!!!!!!!!!!!!!
#directorio_inicial1 = r'D:\sentirobots\Datasets\AffWild2\cropped_images\Caras_AffWild\batch2\Parte 4\batch2' # Carpeta actual
#directorio_inicial1 = r'D:\sentirobots\Datasets\AffWild2\cropped_images\Caras_AffWild\batch2\Parte 3\batch22' # Carpeta actual
directorio_inicial1 = r'D:\sentirobots\Datasets\AffWild2\cropped_images\Caras_AffWild\batch1\Parte 1\batch1' # Carpeta actual
#directorio_inicial1 = r'D:\sentirobots\Datasets\AffWild2\cropped_images\Caras_AffWild\batch1\Parte 2\batch12' # Carpeta actual
pasa=[]
archivos1 = []
carpeta_base1=[]
puntos1=[]
filtro1=[]
file1=[]
r=[]
b=[]
co=[]
e=[]
f=[]
d=[]
contadorcio=0
for raiz, carpetas, archivos_en_carpeta in os.walk(directorio_inicial1):
    for archivo in archivos_en_carpeta:
        if os.path.isfile(os.path.join(raiz, archivo)) and archivo.lower().endswith(".jpg"):
            ruta_completa1 = os.path.join(raiz, archivo)
            archivos1.append(ruta_completa1)
            re=evaluate_resolucion(ruta_completa1)
            r.append(re)
            br=evaluate_brillo(ruta_completa1)
            b.append(br)
            con=evaluate_contraste(ruta_completa1)
            co.append(con)
            en=evaluate_entropia(ruta_completa1)
            e.append(en)
            de=detect_faces(ruta_completa1)
            f.append(de)
            di=difuminacion(ruta_completa1)
            d.append(di)
            if evaluate_image_qualitys(ruta_completa1):
                #print("Verdadero")
                pasa.append(1)
                if br>230 and con<20:
                    pasa[-1] = 2
                    print(ruta_completa1)
                    print("Transformacion brillo contraste o entropia")
                    print("brillo")
                    print(br)
                    print("contraste")
                    print(con)
            else:
                #print("Falso")
                pasa.append(0)
                if len(archivos1) > 1:
                    carpeta_anterior = os.path.dirname(archivos1[-2])
                    carpeta_actual = os.path.dirname(archivos1[-1])
                    a1 = os.path.join(archivos1[-2])
                    a2 = os.path.join(archivos1[-1])
                    if carpeta_anterior == carpeta_actual:
                        if pasa[-2] >= 1:
                            #PONER EL HOG CON LA SIMILITUD DEL COSENO
                            if (sim_coseno(a1, a2)>0.9 and di>4.5 and de>0) or (de>9 and sim_coseno(a1, a2)>0.9) or (de>5 and sim_coseno(a1, a2)>0.9 and di>5 and con>4 and br>14 and en>4):
                                #print(a2)
                                #print("Transformacion")
                                pasa[-1] = 2
            contadorcio=contadorcio+1
            if contadorcio % 1000 == 0:
                print(contadorcio)
            #detectar_rostros_hog(ruta_completa1)
            
#Parte 1 854.564 archivos
#Parte 2 799.322 archivos
#Parte 3 560.506 archivos
#Parte 4 448.501 archivos

C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_8892\3087785669.py:57: FutureWarning: `selem` is a deprecated argument name for `entropy`. It will be removed in version 1.0. Please use `footprint` instead.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_8892\3087785669.py:57: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  entropy = rank.entropy(im1, selem=selem)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000


C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_8892\715643837.py:13: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_8892\715643837.py:15: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_8892\3087785669.py:57: FutureWarning: `selem` is a deprecated argument name for `entropy`. It will be removed in version 1.0. Please use `footprint` instead.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_8892\3087785669.py:57: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters

32000
33000
34000
35000
36000
37000
38000
39000
40000


C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_8892\715643837.py:13: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_8892\715643837.py:15: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_8892\3087785669.py:57: FutureWarning: `selem` is a deprecated argument name for `entropy`. It will be removed in version 1.0. Please use `footprint` instead.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_8892\3087785669.py:57: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters

...

339000
340000
341000
342000
343000
344000
345000
346000
347000
348000
349000
350000


C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_8892\715643837.py:13: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_8892\715643837.py:15: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_8892\3087785669.py:57: FutureWarning: `selem` is a deprecated argument name for `entropy`. It will be removed in version 1.0. Please use `footprint` instead.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_8892\3087785669.py:57: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters

351000


C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_8892\715643837.py:13: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_8892\715643837.py:15: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_8892\3087785669.py:57: FutureWarning: `selem` is a deprecated argument name for `entropy`. It will be removed in version 1.0. Please use `footprint` instead.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_8892\3087785669.py:57: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters

352000
353000
354000
355000
356000
357000
358000
359000
360000
361000
362000


In [5]:
#Real
archiv={"RutaArchivo": archivos1}
df= pd.DataFrame(archiv)
df["Filtro"] = pasa
df["resolucion"] = r
df["brillo"] = b
df["contraste"] = co
df["entropia"] = e
df["p rostro"] = f
df["difumin"] = d
df

,RutaArchivo,Filtro,resolucion,brillo,contraste,entropia,p rostro,difumin
0,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,192.392250,34.541026,5.110318,43,34.541026
1,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,192.751991,30.861124,5.095011,44,30.861124
2,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,193.072953,33.624827,5.094355,34,33.624827
3,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,192.244569,35.497410,5.120029,38,35.497410
4,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,192.371022,32.788471,5.102438,38,32.788471
...,...,...,...,...,...,...,...,...
362810,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,147.312776,142.254008,5.780613,91,142.254008
362811,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,146.621343,132.808878,5.767261,102,132.808878
362812,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,147.366758,138.814116,5.748716,102,138.814116
362813,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,146.743213,146.583364,5.756821,110,146.583364


In [6]:
# Evaluar si los valores de las columnas cumplen con la condición utilizando un OR
condicion1 = df['resolucion'] == True
condicion2 = df['brillo'] < 14
condicion3 = df['contraste'] < 3.5
condicion4 = df['entropia'] < 3
condicion5 = df['p rostro'] <= 3
condicion6 = df['difumin'] <= 5.7

df['Falla en'] = pd.concat([condicion1, condicion2, condicion3, condicion4, condicion5, condicion6], axis=1).sum(axis=1)
df

,RutaArchivo,Filtro,resolucion,brillo,contraste,entropia,p rostro,difumin,Falla en
0,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,192.392250,34.541026,5.110318,43,34.541026,0
1,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,192.751991,30.861124,5.095011,44,30.861124,0
2,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,193.072953,33.624827,5.094355,34,33.624827,0
3,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,192.244569,35.497410,5.120029,38,35.497410,0
4,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,192.371022,32.788471,5.102438,38,32.788471,0
...,...,...,...,...,...,...,...,...,...
362810,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,147.312776,142.254008,5.780613,91,142.254008,0
362811,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,146.621343,132.808878,5.767261,102,132.808878,0
362812,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,147.366758,138.814116,5.748716,102,138.814116,0
362813,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,146.743213,146.583364,5.756821,110,146.583364,0


In [7]:
df.describe()

,Filtro,brillo,contraste,entropia,p rostro,difumin,Falla en
count,362815.000000,362815.000000,362815.000000,362815.000000,362815.000000,362815.000000,362815.000000
mean,0.994184,138.591317,86.031738,5.143527,104.894996,86.031738,0.013740
std,0.113255,33.675309,82.813950,0.665244,147.654165,82.813950,0.128475
min,0.000000,1.587658,0.212336,0.575515,0.000000,0.212336,0.000000
25%,1.000000,117.568846,32.621711,4.676542,44.000000,32.621711,0.000000
50%,1.000000,140.694672,54.572231,5.092951,75.000000,54.572231,0.000000
75%,1.000000,162.780565,107.330321,5.615620,125.000000,107.330321,0.000000
max,2.000000,248.303012,1963.211244,6.813960,4469.000000,1963.211244,5.000000


In [8]:
cont = df['Filtro'].value_counts()
print(cont)

1    358149
0      3388
2      1278
Name: Filtro, dtype: int64


In [9]:
conte = df['Falla en'].value_counts()
print(conte)

0    358152
1      4469
2       132
4        26
5        20
3        16
Name: Falla en, dtype: int64


In [10]:
df_fallas = df[df['Filtro'] == 0].copy()
df_buenas = df[df['Filtro'] == 1].copy()
df_recupera2 = df[df['Filtro'] == 2].copy()
df_perfe = df[df['Filtro'] >= 1].copy()

In [11]:
df_fallas

,RutaArchivo,Filtro,resolucion,brillo,contraste,entropia,p rostro,difumin,Falla en
31235,D:\sentirobots\Datasets\AffWild2\cropped_image...,0,False,67.098053,11.433167,3.714468,1,11.433167,1
31236,D:\sentirobots\Datasets\AffWild2\cropped_image...,0,False,61.379252,7.547528,3.492343,1,7.547528,1
31237,D:\sentirobots\Datasets\AffWild2\cropped_image...,0,False,61.377664,7.544867,3.489700,1,7.544867,1
40939,D:\sentirobots\Datasets\AffWild2\cropped_image...,0,False,83.051640,13.512302,4.224524,2,13.512302,1
53818,D:\sentirobots\Datasets\AffWild2\cropped_image...,0,False,69.651540,10.037743,4.131919,3,10.037743,1
...,...,...,...,...,...,...,...,...,...
338659,D:\sentirobots\Datasets\AffWild2\cropped_image...,0,False,4.793229,0.246709,0.928033,0,0.246709,5
344752,D:\sentirobots\Datasets\AffWild2\cropped_image...,0,False,67.676975,6.966401,3.026145,3,6.966401,1
350662,D:\sentirobots\Datasets\AffWild2\cropped_image...,0,False,101.625423,7.644347,3.190658,2,7.644347,1
350760,D:\sentirobots\Datasets\AffWild2\cropped_image...,0,False,153.798944,26.420542,4.964862,3,26.420542,1


In [12]:
contefalla = df_fallas['Falla en'].value_counts()
print(contefalla)

1    3204
2     122
4      26
5      20
3      16
Name: Falla en, dtype: int64


In [13]:
df_fallas.describe()

,Filtro,brillo,contraste,entropia,p rostro,difumin,Falla en
count,3388.0,3388.000000,3388.000000,3388.000000,3388.000000,3388.000000,3388.000000
mean,0.0,112.493004,299.454091,5.520014,41.672373,299.454091,1.092090
std,0.0,42.807671,260.384487,1.318333,81.597347,260.384487,0.458257
min,0.0,1.587658,0.212336,0.575515,0.000000,0.212336,1.000000
25%,0.0,85.950823,29.598874,4.985209,12.000000,29.598874,1.000000
50%,0.0,109.575660,311.591355,6.117221,22.000000,311.591355,1.000000
75%,0.0,142.685640,432.753591,6.467899,30.000000,432.753591,1.000000
max,0.0,248.303012,1631.149570,6.755228,615.000000,1631.149570,5.000000


In [14]:
df_buenas

,RutaArchivo,Filtro,resolucion,brillo,contraste,entropia,p rostro,difumin,Falla en
0,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,192.392250,34.541026,5.110318,43,34.541026,0
1,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,192.751991,30.861124,5.095011,44,30.861124,0
2,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,193.072953,33.624827,5.094355,34,33.624827,0
3,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,192.244569,35.497410,5.120029,38,35.497410,0
4,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,192.371022,32.788471,5.102438,38,32.788471,0
...,...,...,...,...,...,...,...,...,...
362810,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,147.312776,142.254008,5.780613,91,142.254008,0
362811,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,146.621343,132.808878,5.767261,102,132.808878,0
362812,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,147.366758,138.814116,5.748716,102,138.814116,0
362813,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,146.743213,146.583364,5.756821,110,146.583364,0


In [15]:
contebuena = df_buenas['Falla en'].value_counts()
print(contebuena)

0    358149
Name: Falla en, dtype: int64


In [16]:
df_buenas.describe()

,Filtro,brillo,contraste,entropia,p rostro,difumin,Falla en
count,358149.0,358149.000000,358149.000000,358149.000000,358149.000000,358149.000000,358149.0
mean,1.0,138.898572,83.437108,5.137003,105.789051,83.437108,0.0
std,0.0,33.499672,74.909802,0.653323,148.185990,74.909802,0.0
min,1.0,26.068997,5.714798,3.000096,4.000000,5.714798,0.0
25%,1.0,118.023498,32.591739,4.675588,45.000000,32.591739,0.0
50%,1.0,141.051314,54.226938,5.085045,76.000000,54.226938,0.0
75%,1.0,162.918556,104.777619,5.601427,125.000000,104.777619,0.0
max,1.0,245.749318,890.305929,6.780744,4469.000000,890.305929,0.0


In [17]:
df_recupera2

,RutaArchivo,Filtro,resolucion,brillo,contraste,entropia,p rostro,difumin,Falla en
54109,D:\sentirobots\Datasets\AffWild2\cropped_image...,2,False,69.821630,11.878533,4.307356,1,11.878533,1
54110,D:\sentirobots\Datasets\AffWild2\cropped_image...,2,False,69.079994,14.865561,4.182388,1,14.865561,1
58228,D:\sentirobots\Datasets\AffWild2\cropped_image...,2,False,99.060025,5.525735,4.007934,31,5.525735,1
58229,D:\sentirobots\Datasets\AffWild2\cropped_image...,2,False,98.943973,5.577199,3.997354,31,5.577199,1
58230,D:\sentirobots\Datasets\AffWild2\cropped_image...,2,False,99.135145,5.419103,3.935885,29,5.419103,1
...,...,...,...,...,...,...,...,...,...
299745,D:\sentirobots\Datasets\AffWild2\cropped_image...,2,False,85.099388,16.964553,4.049832,3,16.964553,1
299746,D:\sentirobots\Datasets\AffWild2\cropped_image...,2,False,84.813831,18.232308,4.054160,2,18.232308,1
328776,D:\sentirobots\Datasets\AffWild2\cropped_image...,2,False,179.508255,5.514508,4.550578,51,5.514508,1
328777,D:\sentirobots\Datasets\AffWild2\cropped_image...,2,False,179.601772,5.341879,4.569014,56,5.341879,1


In [18]:
conterecu = df_recupera2['Falla en'].value_counts()
print(conterecu)

1    1265
2      10
0       3
Name: Falla en, dtype: int64


In [19]:
df_recupera2.describe()

,Filtro,brillo,contraste,entropia,p rostro,difumin,Falla en
count,1278.0,1278.000000,1278.000000,1278.000000,1278.000000,1278.000000,1278.000000
mean,2.0,121.672819,247.369179,5.973723,21.947574,247.369179,1.005477
std,0.0,21.730695,214.685579,0.581413,12.675131,214.685579,0.100748
min,2.0,34.100529,5.148249,2.851555,1.000000,5.148249,0.000000
25%,2.0,110.949916,108.534390,5.856958,13.000000,108.534390,1.000000
50%,2.0,122.778863,160.525233,5.976610,21.000000,160.525233,1.000000
75%,2.0,131.140473,341.263293,6.241813,29.000000,341.263293,1.000000
max,2.0,231.590254,1963.211244,6.813960,66.000000,1963.211244,2.000000


In [20]:
df_perfe

,RutaArchivo,Filtro,resolucion,brillo,contraste,entropia,p rostro,difumin,Falla en
0,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,192.392250,34.541026,5.110318,43,34.541026,0
1,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,192.751991,30.861124,5.095011,44,30.861124,0
2,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,193.072953,33.624827,5.094355,34,33.624827,0
3,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,192.244569,35.497410,5.120029,38,35.497410,0
4,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,192.371022,32.788471,5.102438,38,32.788471,0
...,...,...,...,...,...,...,...,...,...
362810,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,147.312776,142.254008,5.780613,91,142.254008,0
362811,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,146.621343,132.808878,5.767261,102,132.808878,0
362812,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,147.366758,138.814116,5.748716,102,138.814116,0
362813,D:\sentirobots\Datasets\AffWild2\cropped_image...,1,False,146.743213,146.583364,5.756821,110,146.583364,0


In [21]:
conteperfe = df_perfe['Falla en'].value_counts()
print(conteperfe)

0    358152
1      1265
2        10
Name: Falla en, dtype: int64


In [22]:
df_perfe.describe()

,Filtro,brillo,contraste,entropia,p rostro,difumin,Falla en
count,359427.000000,359427.000000,359427.000000,359427.000000,359427.000000,359427.000000,359427.000000
mean,1.003556,138.837323,84.019994,5.139978,105.490940,84.019994,0.003575
std,0.059523,33.480842,76.488512,0.654977,148.008393,76.488512,0.060150
min,1.000000,26.068997,5.148249,2.851555,1.000000,5.148249,0.000000
25%,1.000000,117.952807,32.624203,4.676445,45.000000,32.624203,0.000000
50%,1.000000,140.913759,54.351366,5.088257,76.000000,54.351366,0.000000
75%,1.000000,162.840640,105.470684,5.606384,125.000000,105.470684,0.000000
max,2.000000,245.749318,1963.211244,6.813960,4469.000000,1963.211244,2.000000


In [23]:
df.to_csv('D:\sentirobots\Datasets\AffWild2\AffWild2-Face-Parte1.csv', index=False)
#df.to_csv('D:\sentirobots\Datasets\AffWild2\AffWild2-Face-Parte2.csv', index=False)
#df.to_csv('D:\sentirobots\Datasets\AffWild2\AffWild2-Face-Parte3.csv', index=False)
#df.to_csv('D:\sentirobots\Datasets\AffWild2\AffWild2-Face-Parte4.csv', index=False)